In [ ]:
import keras
import tensorflow as tf
import pandas as pd
import crystal_loader
import numpy as np


In [ ]:
def make_dataset(mini, maxi, maxsub, num):
    numbers = []
    y = []
    features = []
    for i in range(num):
        arr1 = np.random.randint(mini, maxi, size=(np.random.randint(1, maxsub)))
        numbers.append(arr1)
        features.append((len(arr1), arr1.mean()))
        y.append(arr1.sum())
    return np.array(features), np.array(y), numbers

def make_easyset(num_instances, min_comps, max_comps):
    comps = [] # features
    pair_sums = [] # true reconstructed
    sums = [] # y

    for i in range(num_instances):
        arr1 = np.random.randint(1, 10, size=(np.random.randint(1, max_comps), 2))
        comps.append(np.array(arr1))
        pair_sums.append(arr1.sum(axis=-1))
        sums.append(arr1.sum())
    return comps, np.array(sums), pair_sums


In [ ]:
class MLP(keras.Model):
    def __init__(self, n_inputs, layers, output_func="linear"):
        super().__init__()
        n_units = 1
        # subnetwork used to evaluate atomic potential contributions, evaluated for each atom in a structure.
        self.subnet = keras.Sequential(layers=[
            keras.Input(shape=(2,))]  # input layer takes in n_inputs number of symmetry function features
            + layers  # hidden layers
            + [keras.layers.Dense(n_units, activation=output_func)])  # output layer returns individual energy contributions
        self.subnet.layers[0].set_weights([np.ones(shape=(2, n_units)), np.zeros(shape=(n_units,))])
        self.num_features = n_inputs

    def call(self, inputs, training=False):
        '''
        feed-forward method for the model
        should have the signature we ultimately want the model to have,
        i.e. for one structure: Tensor[StructureFeatures] -> Energy_total

        which then for multiple structures:
            Tensor[Tensor[StructureFeatures]] -> Tensor[Energy_total]

        wherein in reality the outermost Tensor is just a list.

        inputs: shape (num_atoms, num_features) tensor

        (None, None, 2)

        features: number of numbers, average of numbers
        '''

        #  subnet.call(Tensor[StructureFeatures]) -> Tensor[EnergyContributions]

        numsum = []

        def process_struct(struct):
            return self.subnet(struct, training=training)

        pairwise_contribs = tf.map_fn(process_struct, inputs, fn_output_signature=tf.RaggedTensorSpec(ragged_rank=0))
        numsum.append(tf.reduce_sum(pairwise_contribs, axis=1))

        return numsum
        # for structure in inputs:
        #     atom_contributions = self.subnet(structure, training=training)
        #     structure_energy = tf.reduce_sum(atom_contributions)
        #     structure_energies.append(structure_energy)

        # return structure_energies

In [ ]:
X, y, y_hidden = make_easyset(8, 2, 6)
foo = MLP(n_inputs=2, layers=[])
Xten = tf.ragged.constant(X, ragged_rank=1, inner_shape=(2,))

print(Xten)
print(Xten.shape)


foo.predict(Xten)


In [ ]:
# X, y, y_contrib = make_dataset(0, 25, 54, 500)


In [ ]:
bpnn = MLP1.fit(
    Xtrain, ytrain,
    batch_size = 20,
    epochs = 40,
    validation_data = (Xval, yval),
    verbose = 10
)